# 毕业论文`GRAD`
## 建立多元线性回归模型，定量评估影响

*`Evan`*\
*`2023-11-16`*
---

In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('../../src/')
from namelist import *
from RandomForest import read_data, rf_importance

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

## 读取数据：将三年数据拼接

In [2]:
lowyears  = [2014,2015,2016]
highyears = [2019,2021,2022]
month     = 'Sep'
region    = 'PRD'
datapath  = datadir + 'Contribution/data/'

df_low  = read_data(lowyears,month,region,datapath)
df_high = read_data(highyears,month,region,datapath)


In [21]:
df_low[df_low['SOL_RAD']<100].index.hour.value_counts()

0     90
1     90
2     90
3     90
4     90
5     90
6     90
7     90
19    90
20    90
21    90
22    90
23    90
18    44
dtype: int64

# 保留白天时刻，去除夜间时刻

In [6]:
diurnal_low  = df_low.between_time('8:00','17:00')
diurnal_high = df_high.between_time('8:00','17:00')

## 建立模型

In [7]:
# 设置变量列表和目标变量
variants = ['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 
            'WSPD10', 'WDIR10','CloudFRAC',
            'NO2', 'VOC', 'PM25', 'ISOP'] # 'PBLH',
target = 'O3'

low_importance = rf_importance(diurnal_low,variants,target)
print('\n===============================================\n')
high_importance = rf_importance(diurnal_high,variants,target)


Mean Squared Error (Random Forest): 235.98486059792899
R-squared (Random Forest): 0.8278199813488083
SFC_TMP: 0.3103251843928805
PM25: 0.13630717295486425
VOC: 0.1190972485210737
WDIR10: 0.1101720529725706
RH: 0.0906134527645986
SOL_RAD: 0.08903222509371947
WSPD10: 0.04556273487548025
NO2: 0.032168340934352474
PRES: 0.025277820070481672
CloudFRAC: 0.023673353456342482
ISOP: 0.017770413963635993


Mean Squared Error (Random Forest): 232.1614435128702
R-squared (Random Forest): 0.8557529752641573
PM25: 0.31758552876456725
SFC_TMP: 0.30644262469030065
SOL_RAD: 0.0935777720904444
WDIR10: 0.07971948743312342
VOC: 0.06426734398964742
NO2: 0.03003289513889454
WSPD10: 0.02984995462942293
RH: 0.022738631399963273
ISOP: 0.020874921384497376
CloudFRAC: 0.01865572753532769
PRES: 0.016255112943811094


## 输出结果

In [22]:
# 合并两组结果，对列重命名
dfoutput = pd.concat([low_importance,high_importance],axis=1)
dfoutput.columns=['Low','High']

# randomforest_concatinate_PRDaveraged
filepath = 'D:/data/Graduation/Contribution/RandomForest_output/'
filename = f'RF_{region}_diurnal.xlsx'
dfoutput.to_excel(filepath + filename,index=True)

# 保留夜间时刻，去除白天时刻

In [23]:
nocturnal_low  = df_low.between_time('18:00','7:00')
nocturnal_high = df_high.between_time('18:00','7:00')

In [27]:
# 设置变量列表和目标变量
variants = ['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 
            'WSPD10', 'WDIR10','CloudFRAC',
            'NO2', 'VOC', 'PM25', 'ISOP'] # 'PBLH',
target = 'O3'

low_importance = rf_importance(nocturnal_low,variants,target)
print('\n===============================================\n')
high_importance = rf_importance(nocturnal_high,variants,target)


Mean Squared Error (Random Forest): 244.72246687646378
R-squared (Random Forest): 0.8037328008857931
RH: 0.24707804312265036
NO2: 0.12042973891121535
WSPD10: 0.12037646762761597
VOC: 0.11775346538880972
PM25: 0.09579234194210147
CloudFRAC: 0.07847701951809223
SFC_TMP: 0.06783974261873522
WDIR10: 0.052472188233266416
PRES: 0.040375215343632004
ISOP: 0.039832991122166976
SOL_RAD: 0.019572786171714342


Mean Squared Error (Random Forest): 240.01530830847997
R-squared (Random Forest): 0.7805373931851759
RH: 0.2659470037964875
PM25: 0.11394967327091982
WSPD10: 0.10930322670051174
CloudFRAC: 0.103246282669609
VOC: 0.10229825162019415
SFC_TMP: 0.07041265506905854
PRES: 0.06895357848421083
WDIR10: 0.06365887457754588
ISOP: 0.052933819690220874
NO2: 0.03761865502589029
SOL_RAD: 0.011677979095351378


In [28]:
# 合并两组结果，对列重命名
dfoutput = pd.concat([low_importance,high_importance],axis=1)
dfoutput.columns=['Low','High']

# randomforest_concatinate_PRDaveraged
filepath = 'D:/data/Graduation/Contribution/RandomForest_output/'
filename = f'RF_{region}_nocturnal.xlsx'
dfoutput.to_excel(filepath + filename,index=True)